In [1]:
import collections
import inspect
from typing import Optional, Any, List
import json 

class ChildNode:
    
    json_fields: List[str] = ["child_id",
                              "child", 
                              "child_name",
                              "parent_id",
                              "parent", 
                              "parent_name", 
                              "color", 
                              "metadata"]
    
    _json_type_enforcer: dict = {'child': str, 
                                 'parent': str}
        
    def __init__(self, 
                 child: Any, 
                 child_id: int, 
                 parent: Optional[Any]=None, 
                 parent_id: Optional[int]=None,
                 color: Optional[str]="#000000",
                 metadata: Optional[str]=None):
        self.child = child
        self.child_name = child.__name__
        self._child_id = child_id
        self.parent = parent
        
        self._parent_id = parent_id
        self.parent_name = None
        self.metadata = metadata
        if parent:
            self.parent_name = parent.__name__
            
        self.color = color
    
    def append_to_metadata(self, metadata: str):
        self._metadata += metadata
        
    @property
    def child_id(self) -> str:
        return str(self._child_id)
    
    @property
    def parent_id(self) -> str:        
        if self._parent_id:            
            return str(self._parent_id)
        return    
        
    def _validate_for_json(self, field):
        f = getattr(self, field)
        if f and field in self._json_type_enforcer:
            f = self._json_type_enforcer[field](f)
        return f
    
    def to_dict(self):
        return {i: self._validate_for_json(i) for i in self.json_fields}
    
        
class ClassGraphTree:
    
    def __init__(self, 
                 baseclass: Any, 
                 funcname: Optional[str]=None, 
                 default_color: Optional[str]= "#000000",
                 func_override_color: Optional[str]= "#ff0000"):
        """
        baseclass: 
            the starting base class to begin mapping from
        funcname: 
            the name of a function to watch for overrides
        default_color: t
            he default outline color of nodes, in any graphviz string
        func_override_color: 
            the outline color of nodes that override funcname, in any graphviz string
        """
        self.baseclass = baseclass
        self.basename: str = baseclass.__name__
        self.funcname = funcname        
        self._nodenum: int = 0
        self._node_list = []        
        self._current_node = 1                
        self._default_color = default_color
        self._override_color = func_override_color
        self.build()
        
    def _get_source_info(self, obj) -> Optional[str]:
        f = getattr(obj, self.funcname)
        if isinstance(f, collections.abc.Callable):
            return f"{inspect.getsourcefile(f)}:{inspect.getsourcelines(f)[1]}"
        return None
    
    def _node_overrides_func(self, child, parent) -> bool:
        childsrc = self._get_source_info(child)
        parentsrc = self._get_source_info(parent)
        if childsrc != parentsrc:            
            return True # it overrides! 
        return False        

    def _get_new_node_color(self, child, parent) -> str:
        if self.funcname and self._node_overrides_func(child, parent):
            return self._override_color
        return self._default_color

    def _get_baseclass_color(self) -> str:
        color = self._default_color        
        if self.funcname:
            f = getattr(self.baseclass, self.funcname)
            class_where_its_defined = f.__qualname__.split('.')[0]
            if self.basename == class_where_its_defined: 
                # then its defined here, use the override color
                color = self._override_color
        return color
        
    
    def check_subclasses(self, parent, parent_id: int, node_i: int) -> int:
        for child in parent.__subclasses__():            
            color = self._get_new_node_color(child, parent)
            new_node = ChildNode(child, 
                                 node_i, 
                                 parent=parent,
                                 parent_id=parent_id,
                                 color=color,
                                 metadata=f"metadata for child {node_i}")
            self._node_list.append(new_node)
            
                
            node_i += 1
            node_i = self.check_subclasses(child, node_i - 1, node_i)
        return node_i
            
            
    def build(self):
        
        # first construct all the nodes        
        
        color = self._get_baseclass_color()        
        self._node_list.append(ChildNode(self.baseclass, 
                                         self._current_node, 
                                         parent=None, 
                                         color=color,
                                         metadata="the base node"))
        self._current_node += 1
        
        _ = self.check_subclasses(self.baseclass, self._current_node - 1, self._current_node)
        
    def to_plain_list(self) -> List[dict]:
        return [node.to_dict() for node in self._node_list]

In [2]:
from yt.utilities.io_handler import BaseIOHandler

c = ClassGraphTree(BaseIOHandler, "_read_particle_selection")
d = c.to_plain_list()
j = json.dumps(d)

In [3]:
# the dict layout
d[:4]

[{'child_id': '1',
  'child': "<class 'yt.utilities.io_handler.BaseIOHandler'>",
  'child_name': 'BaseIOHandler',
  'parent_id': None,
  'parent': None,
  'parent_name': None,
  'color': '#ff0000',
  'metadata': 'the base node'},
 {'child_id': '2',
  'child': "<class 'yt.utilities.io_handler.BaseParticleIOHandler'>",
  'child_name': 'BaseParticleIOHandler',
  'parent_id': '1',
  'parent': "<class 'yt.utilities.io_handler.BaseIOHandler'>",
  'parent_name': 'BaseIOHandler',
  'color': '#000000',
  'metadata': 'metadata for child 2'},
 {'child_id': '3',
  'child': "<class 'yt.frontends.adaptahop.io.IOHandlerAdaptaHOPBinary'>",
  'child_name': 'IOHandlerAdaptaHOPBinary',
  'parent_id': '2',
  'parent': "<class 'yt.utilities.io_handler.BaseParticleIOHandler'>",
  'parent_name': 'BaseParticleIOHandler',
  'color': '#000000',
  'metadata': 'metadata for child 3'},
 {'child_id': '4',
  'child': "<class 'yt.frontends.ahf.io.IOHandlerAHFHalos'>",
  'child_name': 'IOHandlerAHFHalos',
  'parent_id

The final list of nodes that comes out has the following fields:

`child_id`: the unique identifier of the current node

`child`: the full class-string of the child

`child_name`: the shortened class name

`parent_id`: the id of the parent to this child class (e.g., the parent to the entry with `child_id='4'` is `BaseParticleIOHandler`), may be empty if the node is the top level class.

`parent`: the class-string of the parent, may be empty

`parent_name`: the short name of the parent, may be empty

`color`: the color of the node, denoting whether or not the class of this node overrides the function of interest

`metadata`: a generic string that might have useful info

there is some data-duplication here (the class-strings could be stored only as `child` attributes and looked up), but having the info in each node might make interactive info easier to pull in.
